In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from pynq import Overlay
overlay = Overlay('./bitstream/LUT16bits.bit')

In [3]:
overlay.ip_dict

{'LUT16bits_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf7f56d0>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'LUT16bits_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '4',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'LUT16bits_LUT16bits_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:LUT16bits:1.0'}}

In [4]:
from pynq import MMIO
ip = MMIO(0x43C00000, 0x10000)

In [5]:
ip.device

### 將 r_LUT  的location輸出值轉為2補數:

In [6]:
def twos_complement_32bit(x):
    """將 32-bit 補數整數轉為有號值"""
    if x >= (1 << 31):
        return x - (1 << 32)
    else:
        return x

#### Test:

In [7]:
location_1 = -14 
remainder_1 = 2499

ip.write_mm(0, location_1)
ip.write_mm(8, remainder_1)


remainder = ip.read_mm(4)
location = ip.read_mm(12)


print(remainder)
print(twos_complement_32bit(location))

902
-12


### 開始則中查表驗證

## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 18613  (30bits), N的範圍: 0 ~ 1073741823 (30bits)

### Define function
* (Not including 2's complement range

In [8]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


In [17]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 16) - 1    ## 
data2 = (1 << 16) - 19
data3 = (1 << 16) - 15
AN = 4547
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)

inputs1的位元數: 29
inputs2的位元數: 29
inputs3的位元數: 29


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [18]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, 24, -24, 25, -25, 26, -26, 27, -27, 28, -28, 29, -29, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, 24, -24, 25, -25, 26, -26, 27, -27, 28, -28, 29, -29, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9

### 產生SEC的模數:

In [19]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 4546, 2, 4545, 4, 4543, 8, 4539, 16, 4531, 32, 4515, 64, 4483, 128, 4419, 256, 4291, 512, 4035, 1024, 3523, 2048, 2499, 4096, 451, 3645, 902, 2743, 1804, 939, 3608, 1878, 2669, 3756, 791, 2965, 1582, 1383, 3164, 2766, 1781, 985, 3562, 1970, 2577, 3940, 607, 3333, 1214, 2119, 2428, 4238, 309, 3929, 618, 3311, 1236]
AWE1的模數:
 [1, 4546, 2, 4545, 4, 4543, 8, 4539, 16, 4531, 32, 4515, 64, 4483, 128, 4419, 256, 4291, 512, 4035, 1024, 3523, 2048, 2499, 4096, 451, 3645, 902, 2743, 1804, 939, 3608, 1878, 2669, 3756, 791, 2965, 1582, 1383, 3164, 2766, 1781, 985, 3562, 1970, 2577, 3940, 607, 3333, 1214, 2119, 2428, 4238, 309, 3929, 618, 3311, 1236]
AWE1的模數是否等於SEC模數: True
SEC的大小: 58


### 產生2-bit AWE (Arithmetic-Weight Error)

In [20]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 297987648
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 297987644
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 297987646
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 297987642
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 297987650
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 297987642
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 297987648
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 297987640
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 297987654
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 297987638
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 297987652
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 297987636
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 297987662
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 297987630
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 297987660
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 297987628
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 297987678
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 297987614
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 297987676
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 297987612
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 297987710
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 297987582
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 297987708
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 297987580
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 297987774
第 24 筆算術錯誤: 129
第 25 筆錯誤資料: 297987518
第 25 筆算術錯誤: -127
第 26 筆錯誤資料: 297987772
第 26 筆算術錯誤: 127
第 27 筆錯誤資料: 29798

第 263 筆錯誤資料: 297979449
第 263 筆算術錯誤: -8196
第 264 筆錯誤資料: 298004033
第 264 筆算術錯誤: 16388
第 265 筆錯誤資料: 297971265
第 265 筆算術錯誤: -16380
第 266 筆錯誤資料: 298004025
第 266 筆算術錯誤: 16380
第 267 筆錯誤資料: 297971257
第 267 筆算術錯誤: -16388
第 268 筆錯誤資料: 298020417
第 268 筆算術錯誤: 32772
第 269 筆錯誤資料: 297954881
第 269 筆算術錯誤: -32764
第 270 筆錯誤資料: 298020409
第 270 筆算術錯誤: 32764
第 271 筆錯誤資料: 297954873
第 271 筆算術錯誤: -32772
第 272 筆錯誤資料: 298053185
第 272 筆算術錯誤: 65540
第 273 筆錯誤資料: 297922113
第 273 筆算術錯誤: -65532
第 274 筆錯誤資料: 298053177
第 274 筆算術錯誤: 65532
第 275 筆錯誤資料: 297922105
第 275 筆算術錯誤: -65540
第 276 筆錯誤資料: 298118721
第 276 筆算術錯誤: 131076
第 277 筆錯誤資料: 297856577
第 277 筆算術錯誤: -131068
第 278 筆錯誤資料: 298118713
第 278 筆算術錯誤: 131068
第 279 筆錯誤資料: 297856569
第 279 筆算術錯誤: -131076
第 280 筆錯誤資料: 298249793
第 280 筆算術錯誤: 262148
第 281 筆錯誤資料: 297725505
第 281 筆算術錯誤: -262140
第 282 筆錯誤資料: 298249785
第 282 筆算術錯誤: 262140
第 283 筆錯誤資料: 297725497
第 283 筆算術錯誤: -262148
第 284 筆錯誤資料: 298511937
第 284 筆算術錯誤: 524292
第 285 筆錯誤資料: 297463361
第 285 筆算術錯誤: -524284
第 286 筆錯誤資料: 

第 516 筆錯誤資料: 566423117
第 516 筆算術錯誤: 268435472
第 517 筆錯誤資料: 29552205
第 517 筆算術錯誤: -268435440
第 518 筆錯誤資料: 566423085
第 518 筆算術錯誤: 268435440
第 519 筆錯誤資料: 29552173
第 519 筆算術錯誤: -268435472
第 520 筆錯誤資料: 297987741
第 520 筆算術錯誤: 96
第 521 筆錯誤資料: 297987613
第 521 筆算術錯誤: -32
第 522 筆錯誤資料: 297987677
第 522 筆算術錯誤: 32
第 523 筆錯誤資料: 297987549
第 523 筆算術錯誤: -96
第 524 筆錯誤資料: 297987805
第 524 筆算術錯誤: 160
第 525 筆錯誤資料: 297987549
第 525 筆算術錯誤: -96
第 526 筆錯誤資料: 297987741
第 526 筆算術錯誤: 96
第 527 筆錯誤資料: 297987485
第 527 筆算術錯誤: -160
第 528 筆錯誤資料: 297987933
第 528 筆算術錯誤: 288
第 529 筆錯誤資料: 297987421
第 529 筆算術錯誤: -224
第 530 筆錯誤資料: 297987869
第 530 筆算術錯誤: 224
第 531 筆錯誤資料: 297987357
第 531 筆算術錯誤: -288
第 532 筆錯誤資料: 297988189
第 532 筆算術錯誤: 544
第 533 筆錯誤資料: 297987165
第 533 筆算術錯誤: -480
第 534 筆錯誤資料: 297988125
第 534 筆算術錯誤: 480
第 535 筆錯誤資料: 297987101
第 535 筆算術錯誤: -544
第 536 筆錯誤資料: 297988701
第 536 筆算術錯誤: 1056
第 537 筆錯誤資料: 297986653
第 537 筆算術錯誤: -992
第 538 筆錯誤資料: 297988637
第 538 筆算術錯誤: 992
第 539 筆錯誤資料: 297986589
第 539 筆算術錯誤: -1056
第 540 筆錯誤資

第 765 筆算術錯誤: -16777088
第 766 筆錯誤資料: 314764733
第 766 筆算術錯誤: 16777088
第 767 筆錯誤資料: 281210301
第 767 筆算術錯誤: -16777344
第 768 筆錯誤資料: 331542205
第 768 筆算術錯誤: 33554560
第 769 筆錯誤資料: 264433341
第 769 筆算術錯誤: -33554304
第 770 筆錯誤資料: 331541949
第 770 筆算術錯誤: 33554304
第 771 筆錯誤資料: 264433085
第 771 筆算術錯誤: -33554560
第 772 筆錯誤資料: 365096637
第 772 筆算術錯誤: 67108992
第 773 筆錯誤資料: 230878909
第 773 筆算術錯誤: -67108736
第 774 筆錯誤資料: 365096381
第 774 筆算術錯誤: 67108736
第 775 筆錯誤資料: 230878653
第 775 筆算術錯誤: -67108992
第 776 筆錯誤資料: 432205501
第 776 筆算術錯誤: 134217856
第 777 筆錯誤資料: 163770045
第 777 筆算術錯誤: -134217600
第 778 筆錯誤資料: 432205245
第 778 筆算術錯誤: 134217600
第 779 筆錯誤資料: 163769789
第 779 筆算術錯誤: -134217856
第 780 筆錯誤資料: 566423229
第 780 筆算術錯誤: 268435584
第 781 筆錯誤資料: 29552317
第 781 筆算術錯誤: -268435328
第 782 筆錯誤資料: 566422973
第 782 筆算術錯誤: 268435328
第 783 筆錯誤資料: 29552061
第 783 筆算術錯誤: -268435584
第 784 筆錯誤資料: 297988413
第 784 筆算術錯誤: 768
第 785 筆錯誤資料: 297987389
第 785 筆算術錯誤: -256
第 786 筆錯誤資料: 297987901
第 786 筆算術錯誤: 256
第 787 筆錯誤資料: 297986877
第 787 筆算

第 1015 筆算術錯誤: -6144
第 1016 筆錯誤資料: 297997885
第 1016 筆算術錯誤: 10240
第 1017 筆錯誤資料: 297981501
第 1017 筆算術錯誤: -6144
第 1018 筆錯誤資料: 297993789
第 1018 筆算術錯誤: 6144
第 1019 筆錯誤資料: 297977405
第 1019 筆算術錯誤: -10240
第 1020 筆錯誤資料: 298006077
第 1020 筆算術錯誤: 18432
第 1021 筆錯誤資料: 297973309
第 1021 筆算術錯誤: -14336
第 1022 筆錯誤資料: 298001981
第 1022 筆算術錯誤: 14336
第 1023 筆錯誤資料: 297969213
第 1023 筆算術錯誤: -18432
第 1024 筆錯誤資料: 298022461
第 1024 筆算術錯誤: 34816
第 1025 筆錯誤資料: 297956925
第 1025 筆算術錯誤: -30720
第 1026 筆錯誤資料: 298018365
第 1026 筆算術錯誤: 30720
第 1027 筆錯誤資料: 297952829
第 1027 筆算術錯誤: -34816
第 1028 筆錯誤資料: 298055229
第 1028 筆算術錯誤: 67584
第 1029 筆錯誤資料: 297924157
第 1029 筆算術錯誤: -63488
第 1030 筆錯誤資料: 298051133
第 1030 筆算術錯誤: 63488
第 1031 筆錯誤資料: 297920061
第 1031 筆算術錯誤: -67584
第 1032 筆錯誤資料: 298120765
第 1032 筆算術錯誤: 133120
第 1033 筆錯誤資料: 297858621
第 1033 筆算術錯誤: -129024
第 1034 筆錯誤資料: 298116669
第 1034 筆算術錯誤: 129024
第 1035 筆錯誤資料: 297854525
第 1035 筆算術錯誤: -133120
第 1036 筆錯誤資料: 298251837
第 1036 筆算術錯誤: 264192
第 1037 筆錯誤資料: 297727549
第 1037 筆算術錯誤: -2600

第 1265 筆錯誤資料: 297889341
第 1265 筆算術錯誤: -98304
第 1266 筆錯誤資料: 298085949
第 1266 筆算術錯誤: 98304
第 1267 筆錯誤資料: 297823805
第 1267 筆算術錯誤: -163840
第 1268 筆錯誤資料: 298282557
第 1268 筆算術錯誤: 294912
第 1269 筆錯誤資料: 297758269
第 1269 筆算術錯誤: -229376
第 1270 筆錯誤資料: 298217021
第 1270 筆算術錯誤: 229376
第 1271 筆錯誤資料: 297692733
第 1271 筆算術錯誤: -294912
第 1272 筆錯誤資料: 298544701
第 1272 筆算術錯誤: 557056
第 1273 筆錯誤資料: 297496125
第 1273 筆算術錯誤: -491520
第 1274 筆錯誤資料: 298479165
第 1274 筆算術錯誤: 491520
第 1275 筆錯誤資料: 297430589
第 1275 筆算術錯誤: -557056
第 1276 筆錯誤資料: 299068989
第 1276 筆算術錯誤: 1081344
第 1277 筆錯誤資料: 296971837
第 1277 筆算術錯誤: -1015808
第 1278 筆錯誤資料: 299003453
第 1278 筆算術錯誤: 1015808
第 1279 筆錯誤資料: 296906301
第 1279 筆算術錯誤: -1081344
第 1280 筆錯誤資料: 300117565
第 1280 筆算術錯誤: 2129920
第 1281 筆錯誤資料: 295923261
第 1281 筆算術錯誤: -2064384
第 1282 筆錯誤資料: 300052029
第 1282 筆算術錯誤: 2064384
第 1283 筆錯誤資料: 295857725
第 1283 筆算術錯誤: -2129920
第 1284 筆錯誤資料: 302214717
第 1284 筆算術錯誤: 4227072
第 1285 筆錯誤資料: 293826109
第 1285 筆算術錯誤: -4161536
第 1286 筆錯誤資料: 302149181
第 1286 筆算術錯誤

第 1515 筆錯誤資料: 291696189
第 1515 筆算術錯誤: -6291456
第 1516 筆錯誤資料: 308473405
第 1516 筆算術錯誤: 10485760
第 1517 筆錯誤資料: 291696189
第 1517 筆算術錯誤: -6291456
第 1518 筆錯誤資料: 304279101
第 1518 筆算術錯誤: 6291456
第 1519 筆錯誤資料: 287501885
第 1519 筆算術錯誤: -10485760
第 1520 筆錯誤資料: 316862013
第 1520 筆算術錯誤: 18874368
第 1521 筆錯誤資料: 283307581
第 1521 筆算術錯誤: -14680064
第 1522 筆錯誤資料: 312667709
第 1522 筆算術錯誤: 14680064
第 1523 筆錯誤資料: 279113277
第 1523 筆算術錯誤: -18874368
第 1524 筆錯誤資料: 333639229
第 1524 筆算術錯誤: 35651584
第 1525 筆錯誤資料: 266530365
第 1525 筆算術錯誤: -31457280
第 1526 筆錯誤資料: 329444925
第 1526 筆算術錯誤: 31457280
第 1527 筆錯誤資料: 262336061
第 1527 筆算術錯誤: -35651584
第 1528 筆錯誤資料: 367193661
第 1528 筆算術錯誤: 69206016
第 1529 筆錯誤資料: 232975933
第 1529 筆算術錯誤: -65011712
第 1530 筆錯誤資料: 362999357
第 1530 筆算術錯誤: 65011712
第 1531 筆錯誤資料: 228781629
第 1531 筆算術錯誤: -69206016
第 1532 筆錯誤資料: 434302525
第 1532 筆算術錯誤: 136314880
第 1533 筆錯誤資料: 165867069
第 1533 筆算術錯誤: -132120576
第 1534 筆錯誤資料: 430108221
第 1534 筆算術錯誤: 132120576
第 1535 筆錯誤資料: 161672765
第 1535 筆算術錯誤: -136314880
第

In [21]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 4546, 1, 4544, 5, 4544, 3, 4542, 9, 4540, 7, 4538, 17, 4532, 15, 4530, 33, 4516, 31, 4514, 65, 4484, 63, 4482, 129, 4420, 127, 4418, 257, 4292, 255, 4290, 513, 4036, 511, 4034, 1025, 3524, 1023, 3522, 2049, 2500, 2047, 2498, 4097, 452, 4095, 450, 3646, 903, 3644, 901, 2744, 1805, 2742, 1803, 940, 3609, 938, 3607, 1879, 2670, 1877, 2668, 3757, 792, 3755, 790, 2966, 1583, 2964, 1581, 1384, 3165, 1382, 3163, 2767, 1782, 2765, 1780, 986, 3563, 984, 3561, 1971, 2578, 1969, 2576, 3941, 608, 3939, 606, 3334, 1215, 3332, 1213, 2120, 2429, 2118, 2427, 4239, 310, 4237, 308, 3930, 619, 3928, 617, 3312, 1237, 3310, 1235, 6, 4545, 2, 4541, 10, 4541, 6, 4537, 18, 4533, 14, 4529, 34, 4517, 30, 4513, 66, 4485, 62, 4481, 130, 4421, 126, 4417, 258, 4293, 254, 4289, 514, 4037, 510, 4033, 1026, 3525, 1022, 3521, 2050, 2501, 2046, 2497, 4098, 453, 4094, 449, 3647, 904, 3643, 900, 2745, 1806, 2741, 1802, 941, 3610, 937, 3606, 1880, 2671, 1876, 2667, 3758, 793, 3754, 789, 2967, 1584, 2963, 158

In [23]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已未去除重複餘數的數量:",np.size(new_R_total))
print("已未去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)

還未去除重複error data的數量: 1682
還未去除重複餘數的數量: 1682
已去除重複error data的數量: 1572
已未去除重複餘數的數量: 1572
已未去除重複AWE的數量: 1572
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 8193, -8191, 8191, -8193, 16385, -16383, 16383, -16385, 32769, -32767, 32767, -32769, 65537, -65535, 65535, -65537, 131073, -131071, 131071, -131073, 262145, -262143, 262143, -262145, 524289, -524287, 524287, -524289, 1048577, -1048575,

### 所有AWE數量: $4*C^{bits}_{2}$ $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/05/27 使用 Vivado HLX IP 驗證
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [24]:
import math
import time
### 給定一個DEC_R, 找錯誤位置h1及h2(包含錯誤正負號)
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]

start_time = time.time()  # 記錄開始時間
for R, error in zip(new_R_total, new_e_total): 
    found = False                                       # 旗標，用來標示是否要跳出 for H 循環 
    for H in range(0,bits1):                              
        for s in range(-1,2,2):                       
            h1 = s*(H+1)
            ip.write_mm(0, h1)
            R1 = ip.read_mm(4)
            R2 = R - R1
            if R2 < 0:
                R2 = AN + R2
            
            ip.write_mm(8, R2)
            h2 = twos_complement_32bit(ip.read_mm(12))                  
            if (h2 != 0):
                s2 = get_sign(h2)                     
                possible_h1.append(h1)                  # 若找不到對應的h2,h2定為0(函數內容已定義)
                possible_h2.append(h2)                  # 代表錯誤可能是三個或以上(multiple errors)
                output.append((error - (s*2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)
                found = True
                break
        if found:
            break
            
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
print(f"執行時間: {elapsed_time:.6f} 秒")
print("更正後的值是否和原data相同:", data1 == output.pop())
print(elapsed_time / np.size(possible_h1))
##print(possible_h1)
##print(possible_h2)
##print("更正後的值:",output)
print(np.size(possible_h1))
print(np.size(possible_h2))
##print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
##print(possible_SE_h1)

執行時間: 3.450736 秒
更正後的值是否和原data相同: True
0.0021951245594267325
1572
1572
